In [81]:
from lusidtools.jupyter_tools import toggle_code

"""Recipe Composer Workflow

Attributes
----------
recipe composers
recipes
valuations
"""

toggle_code("Toggle Docstring")

# Composing Configuration Recipes
Configuration Recipes are used for valuation, look-through and more. Its purpose is to allow user to be able to specify what market data to use, how to do lookthrough, what model to select and so on. Due to how much it is covering it can get quiet big and bespoke and is reasonable for a team or an individual to have their own. Problem arises when a user wants a recipe that is just like an already upserted recipe except with some small changes. Perhaps a user wants to combine market data rules from two recipes, prioritising ones over the other. In addition, the user will likely want their recipe to change should the recipe they depend on change also.

This is where Recipe Composer comes in. It is made of scope-code (identification) and a list of operations. Operations are allowing you the user to state how you want the Configuration Recipe to be composed. You could say "I want to start of with recipe A", that is one operation, then perhaps "I want to add market rules from recipe B", that is another operation and so on. Recipe Composer supports CRUD operations just like Configuration Recipes, moreover you can use Recipe Composers wherever Configuration Recipe is used and it will create the desired recipe and use that without you having to do anything differently.

In this notebook, we first present some Recipe Composers, showing some examples, we then demonstrate how a user can get a Configuration Recipe out of a Recipe Composer that is either upserted or inline. Following that we show how one can use a Recipe Composer in Valuation.

In [82]:
# Import LUSID libraries
import lusid as lu
import lusid.models as lm
from lusidjam.refreshing_token import RefreshingToken
from datetime import datetime, timedelta
import pytz
import pandas as pd
import os
import json
from typing import Any
# Settings and utility functions to display objects and responses more clearly.
pd.set_option('float_format', '{:,.4f}'.format)

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory = lu.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', api_factory.build(lu.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID SDK Version:  0.6.12698.0


In [83]:
# Initiate the LUSID APIs required for the notebook
configuration_recipe_api = api_factory.build(lu.api.ConfigurationRecipeApi)
instruments_api = api_factory.build(lu.api.InstrumentsApi)
transaction_portfolios_api = api_factory.build(lu.api.TransactionPortfoliosApi)
quotes_api = api_factory.build(lu.api.QuotesApi)
aggregation_api = api_factory.build(lu.api.AggregationApi)

In [84]:
# Helper functions 
def get_recipe_scope(recipe_name: str) -> str:
    return f"{recipe_name}-recipe-composer-scope-n"
def get_recipe_code(recipe_name: str) -> str:
    return f"{recipe_name}-recipe-composer-code-n"
def upsert_recipe(recipe: lu.ConfigurationRecipe) -> None:
    # Upsert configuration recipe
    response = configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=lm.UpsertRecipeRequest(configuration_recipe=recipe)
    )
    print(response)
    #print_json(str(response))
def resolve_recipe_composer(recipe_composer: lu.RecipeComposer) -> lu.ConfigurationRecipe:
    response: lu.GetRecipeResponse = configuration_recipe_api.get_recipe_composer_resolved_inline(
        upsert_recipe_composer_request = lm.UpsertRecipeComposerRequest(
            recipe_composer = recipe_composer
        )
    )
    return response.value
def pprint(openapi_obj: Any) -> None:
    resp = openapi_obj.to_dict()
    print(json.dumps(resp, indent=4))
def create_fx_option(strike, dom_ccy, fgn_ccy, start_date, maturity_date, settlement_date, is_call, is_fx_delivery = True, is_payoff_digital = False, exercise_type = "European"):

    return lm.FxOption(
        strike = strike,
        dom_ccy = dom_ccy,
        fgn_ccy = fgn_ccy,
        start_date = start_date,
        option_maturity_date = maturity_date,
        option_settlement_date = settlement_date,
        is_call_not_put = is_call,
        is_delivery_not_cash = is_fx_delivery,
        is_payoff_digital = is_payoff_digital,
        instrument_type = "FxOption",
        dom_amount = 1,
        exercise_type = exercise_type
    )
market_supplier = "Lusid"

# Will use this scope and code for last recipe composer and then in valuation
recipe_composer_scope = "aVivaldiScope"
recipe_composer_code = "aVivaldiCode"

# Creating Configuration Recipes
In order to demonstrate how Recipe Composer can be used to build a new Recipe lets first create some Recipes in a typical way and then use them to create new Recipes using Recipe Composers. 

In [85]:
recipe1 = lm.ConfigurationRecipe(
    scope = get_recipe_scope("recipe1"),
    code = get_recipe_code("recipe1"),
    market=lm.MarketContext(
        market_rules=[
            lm.MarketDataKeyRule(
                key="FX.CurrencyPair.*",
                supplier=market_supplier,
                data_scope=recipe_composer_scope,
                quote_type="Rate",
                field="mid",
                quote_interval="100D"
            ),
            lm.MarketDataKeyRule(
                key="FXVol.*.*.*",
                supplier=market_supplier,
                data_scope=recipe_composer_scope,
                price_source=market_supplier,
                quote_type="Price",
                field="mid",
                quote_interval="100D",
            ),
            lm.MarketDataKeyRule(
                key="Rates.*.*",
                supplier=market_supplier,
                data_scope=recipe_composer_scope,
                price_source=market_supplier,
                quote_type="Price",
                field="mid",
                quote_interval="100D",
            ),
        ],
        options=lm.MarketOptions(
            default_scope = recipe_composer_scope,
            attempt_to_infer_missing_fx=True
        ),
    ),
    pricing = lm.PricingContext(
        model_rules = [
            lm.VendorModelRule(
                supplier = "Lusid",
                model_name = "Discounting",
                instrument_type = "InflationSwap",
            ),
            lm.VendorModelRule(
                supplier = "Lusid",
                model_name = "BlackScholes",
                instrument_type = "FxOption",
            ),
            lm.VendorModelRule(
                supplier = "Lusid",
                model_name = "SimpleStatic",
                instrument_type = "FixedLeg",
            )
        ],
    ),
)
upsert_recipe(recipe1)

{'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://fbn-ci.lusid.com/app/insights/logs/0HN1TROVBS4AC:000000A6',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'value': datetime.datetime(2024, 3, 6, 13, 57, 46, 695099, tzinfo=tzutc())}


In [86]:
recipe2 = lm.ConfigurationRecipe(
    scope = get_recipe_scope("recipe2"),
    code = get_recipe_code("recipe2"),
    market= lm.MarketContext(
        market_rules=[
            lm.MarketDataKeyRule(
                key="Quote.RIC.AMZN",
                supplier="Client",
                data_scope="NotUsed",
                quote_type="Price",
                source_system="SimulatedData/Constant/136.01"
            ),
        ]
    ),
    pricing = lm.PricingContext(
        model_rules = [
            lm.VendorModelRule(
                supplier = "Lusid",
                model_name = "BlackScholes",
                instrument_type = "FxOption",
                address_key_filters = [
                    lm.AddressKeyFilter(left="Instrument/Features/ExerciseType", operator="eq", right=lm.ResultValueString(value="European", result_value_type="ResultValueString"))
                ] 
            ),
            lm.VendorModelRule(
                supplier = "Lusid",
                model_name = "ConstantTimeValueOfMoney",
                instrument_type = "ComplexBond",
            )
        ],
    ),
)
upsert_recipe(recipe2)

{'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://fbn-ci.lusid.com/app/insights/logs/0HN1TROVBS4B9:0000005C',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'value': datetime.datetime(2024, 3, 6, 13, 57, 47, 13312, tzinfo=tzutc())}


# Recipe Composing
Recipe Composer is very flexible, user has a lot of freedom to set up their Recipes the way that fits their use-case, it can however be challenging to get it right on the first attempt. Because of this we have made a GetRecipeComposerResolvedInline endpoint (`configuration_recipe_api.get_recipe_composer_resolved_inline` in the SDK which here is called via `resolve_recipe_composer`), it allows a user to see what the Recipe Composer would provide without actually upserting it, allowing user to do fast prototyping and getting the outcome they want fast.

In [87]:
recipe_composer1 = lm.RecipeComposer(
    code = "vivaldiCode1",
    scope = "vivaldiScope1",
    operations = [
        lm.RecipeBlock(
            op = "Insert",
            path = "$",
            value = lm.RecipeValue(from_recipe=lm.FromRecipe(scope=get_recipe_scope("recipe1"), code=get_recipe_code("recipe1")))
        ),
        lm.RecipeBlock(
            op = "Prepend",
            path = "Pricing.ModelRules",
            value = lm.RecipeValue(from_recipe=lm.FromRecipe(scope=get_recipe_scope("recipe2"), code=get_recipe_code("recipe2")))
        )
    ]
)
resolved_recipe1 = resolve_recipe_composer(recipe_composer1)

pprint(resolved_recipe1)

{
    "scope": "vivaldiScope1",
    "code": "vivaldiCode1",
    "market": {
        "market_rules": [
            {
                "key": "FX.CurrencyPair.*",
                "supplier": "Lusid",
                "data_scope": "aVivaldiScope",
                "quote_type": "Rate",
                "field": "mid",
                "quote_interval": "100D",
                "as_at": null,
                "price_source": "",
                "mask": null,
                "source_system": "Lusid"
            },
            {
                "key": "FXVol.*.*.*",
                "supplier": "Lusid",
                "data_scope": "aVivaldiScope",
                "quote_type": "Price",
                "field": "mid",
                "quote_interval": "100D",
                "as_at": null,
                "price_source": "Lusid",
                "mask": null,
                "source_system": "Lusid"
            },
            {
                "key": "Rates.*.*",
                "supplier": "Lusid

Its nice to get data from other recipe, it is also possible to provide the desired part of recipe inline. Above we saw operations like "Insert" and "Prepend", there are 5 operations in total, namely
"Insert", "Update", "Remove", "Prepend", "Append". Do note Insert can only be done on parts which don't already exist, meanwhile Update and Remove can only be ran on parts that already exist. Prepend and Append can only be used on arrays.

In [88]:
recipe_composer2 = lm.RecipeComposer(
    code = "vivaldiCode2",
    scope = "vivaldiScope2",
    operations = [
        lm.RecipeBlock(
            op = "Insert",
            path = "$",
            value = lm.RecipeValue(from_recipe=lm.FromRecipe(scope=get_recipe_scope("recipe1"), code=get_recipe_code("recipe1")))
        ),
        lm.RecipeBlock(
            op = "Append",
            path = "Pricing.ModelRules",
            value = lm.RecipeValue(as_json="[{\"Supplier\":\"Lusid\",\"ModelName\":\"SimpleStatic\",\"InstrumentType\":\"TotalReturnSwap\",\"Parameters\":null,\"ModelOptions\":{\"ModelOptionsType\":2},\"InstrumentId\":\"\",\"AddressKeyFilters\":[]}]")
        ),
        lm.RecipeBlock(
            op = "Prepend",
            path = "Pricing.ModelRules",
            value = lm.RecipeValue(from_recipe=lm.FromRecipe(scope=get_recipe_scope("recipe2"), code=get_recipe_code("recipe2")))
        )
    ]
)
resolved_recipe2 = resolve_recipe_composer(recipe_composer2)

# To reduce confusion, only looking at model rules 
print(resolved_recipe2.pricing.model_rules)

[{'address_key_filters': [{'left': 'Instrument/Features/ExerciseType',
                          'operator': 'eq',
                          'right': {'result_value_type': 'ResultValueString',
                                    'value': 'European'}}],
 'instrument_id': '',
 'instrument_type': 'FxOption',
 'model_name': 'BlackScholes',
 'model_options': {'model_options_type': 'EmptyModelOptions'},
 'parameters': '',
 'supplier': 'Lusid'}, {'address_key_filters': [],
 'instrument_id': '',
 'instrument_type': 'ComplexBond',
 'model_name': 'ConstantTimeValueOfMoney',
 'model_options': {'model_options_type': 'EmptyModelOptions'},
 'parameters': '',
 'supplier': 'Lusid'}, {'address_key_filters': [],
 'instrument_id': '',
 'instrument_type': 'InflationSwap',
 'model_name': 'Discounting',
 'model_options': {'model_options_type': 'EmptyModelOptions'},
 'parameters': '',
 'supplier': 'Lusid'}, {'address_key_filters': [],
 'instrument_id': '',
 'instrument_type': 'FxOption',
 'model_name': 'Blac

We can modify the rules should we wish using Update operation. Furthermore the path attribute on each operation accepts star notation on arrays so `Pricing.ModelRules.[*]` would refer to each element in the array of ModelRules.

In [89]:

recipe_composer3 = lm.RecipeComposer(
    code = recipe_composer_code,
    scope = recipe_composer_scope,
    operations = [
        # Use recipe1 as my starting point
        lm.Operation(
            op = "Insert",
            path = "$",
            value = lm.RecipeValue(from_recipe=lm.FromRecipe(scope=get_recipe_scope("recipe1"), code=get_recipe_code("recipe1")))
        ),
        # Append (low priority) my specific model rule
        lm.RecipeBlock(
            op = "Append",
            path = "Pricing.ModelRules",
            value = lm.RecipeValue(as_json="[{\"Supplier\":\"Lusid\",\"ModelName\":\"SimpleStatic\",\"InstrumentType\":\"TotalReturnSwap\",\"Parameters\":null,\"ModelOptions\":{\"ModelOptionsType\":2},\"InstrumentId\":\"\",\"AddressKeyFilters\":[]}]")
        ),
        # Prepend (prioritise) model rules form recipe 2
        lm.RecipeBlock(
            op = "Prepend",
            path = "Pricing.ModelRules",
            value = lm.RecipeValue(from_recipe=lm.FromRecipe(scope=get_recipe_scope("recipe2"), code=get_recipe_code("recipe2")))
        ),
        # Update all model rules I have to use ConstantTimeValueOfMoney
        lm.RecipeBlock(
            op = "Update",
            path = "Pricing.ModelRules.[*].ModelName",
            value = lm.RecipeValue(as_string="ConstantTimeValueOfMoney")
        ),
    ]
)
resolved_recipe3 = resolve_recipe_composer(recipe_composer3)

# To reduce confusion, only looking at model rules 
print(resolved_recipe3.pricing.model_rules)

[{'address_key_filters': [{'left': 'Instrument/Features/ExerciseType',
                          'operator': 'eq',
                          'right': {'result_value_type': 'ResultValueString',
                                    'value': 'European'}}],
 'instrument_id': '',
 'instrument_type': 'FxOption',
 'model_name': 'ConstantTimeValueOfMoney',
 'model_options': {'model_options_type': 'EmptyModelOptions'},
 'parameters': '',
 'supplier': 'Lusid'}, {'address_key_filters': [],
 'instrument_id': '',
 'instrument_type': 'ComplexBond',
 'model_name': 'ConstantTimeValueOfMoney',
 'model_options': {'model_options_type': 'EmptyModelOptions'},
 'parameters': '',
 'supplier': 'Lusid'}, {'address_key_filters': [],
 'instrument_id': '',
 'instrument_type': 'InflationSwap',
 'model_name': 'ConstantTimeValueOfMoney',
 'model_options': {'model_options_type': 'EmptyModelOptions'},
 'parameters': '',
 'supplier': 'Lusid'}, {'address_key_filters': [],
 'instrument_id': '',
 'instrument_type': 'FxOpti

Do note the Update operation at the very end will replace all of the model rules in the recipe at the time of that operation, so all the model rules that came from the 3 operations above will have their `ModelName` changed to `ConstantTimeValueOfMoney`.

Having figured out what Recipe Composer we want, we can now upsert it. Just as Recipes it supports the standard CRUD operations. After upserting we can always get it back using the GetRecipeComposer, in addition to that we can get it in the resolved Recipe form using GetDerivedRecipe endpoint. GetDerivedRecipe extends the functionality of GetConfigurationRecipe endpoint, returning a Recipe if scope and code for a Recipe is given, and if scope and code is given for RecipeComposer it resolves it to a Recipe and returns that.

In [90]:
upsert = configuration_recipe_api.upsert_recipe_composer(upsert_recipe_composer_request= lu.UpsertRecipeComposerRequest(recipe_composer=recipe_composer3))

# Get back Recipe Composer
recipe_composer_back = configuration_recipe_api.get_recipe_composer(scope=recipe_composer_scope, code=recipe_composer_code)
print(recipe_composer_back)

# Get Back Recipe that is resolved from our Recipe Composer
recipe_back = configuration_recipe_api.get_derived_recipe(scope=recipe_composer_scope, code=recipe_composer_code)
print(recipe_back)

{'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://fbn-ci.lusid.com/app/insights/logs/0HN1TRO0VIQAD:00000067',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'value': {'code': 'aVivaldiCode',
           'operations': [{'op': 'Insert',
                           'path': '$',
                           'value': {'as_json': None,
                                     'as_string': None,
                                     'from_recipe': {'code': 'recipe1-recipe-composer-code-n',
                                                     'scope': 'recipe1-recipe-composer-scope-n'}}},
                          {'op': 'Append',
                           'path': 'Pricing.ModelRules',
                           'value': {'as_json': '[{"Supplier":"Lusid","ModelName":"SimpleStatic","InstrumentType":"TotalReturnSwap","Parameters":null,"ModelOptions":

# Recipe Composer Used In Valuation
As already mentioned, Recipe Composer seamlessly integrates wherever Configuration Recipe does, making a very easy experience.

In [91]:
# Create portfolio
portfolio_code = "recipe-composer-pf"
scope = "someScope-recipe-composer"
try:
    transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="EUR",
            created="2010-01-01",
            instrument_scopes=[scope]
        ),
    )

except lu.ApiException as e:
    print(e.body)

# Create instrument definition
trade_date = datetime(2021, 1, 5, tzinfo=pytz.utc)
start_date = trade_date
maturity_date = trade_date + timedelta(days = 400)
settlement_date = maturity_date + timedelta(days = 2)
strike = 1.19
name = "EUR/USD FX Option " + maturity_date.strftime("%m/%d/%Y")  + " " + str(strike)
identifier = "EURUSDOptionDemo"
dom_ccy = "EUR"
fgn_ccy = "USD"

# Create Instrument
instrument_definition = create_fx_option(
    strike = strike,
    dom_ccy = dom_ccy,
    fgn_ccy = fgn_ccy,
    start_date = start_date,
    maturity_date = maturity_date,
    settlement_date = settlement_date,
    is_call = True,
    is_fx_delivery = True,
    is_payoff_digital = False,
    exercise_type = "European"
)

# Upsert instrument
instruments_api.upsert_instruments(
    request_body={
        identifier: lm.InstrumentDefinition(
            name="myInstrument",
            identifiers={
                "ClientInternal": lm.InstrumentIdValue(value=identifier)
            },
            definition=instrument_definition,
        )
    },
    scope = scope
)

# Create and Upsert Transaction
premium = 0.03
option_version = 1
deal_2_id = "TXN001"
settle_days = 2
units = 50

opt_txn = lm.TransactionRequest(
    transaction_id= deal_2_id,
    type="Sell",
    instrument_identifiers={"Instrument/default/ClientInternal": identifier},
    transaction_date=trade_date.isoformat(),
    settlement_date=(trade_date + timedelta(days = settle_days)).isoformat(),
    units=units,
    transaction_price=lm.TransactionPrice(price=premium,type="Price"),
    total_consideration=lm.CurrencyAndAmount(amount=premium*units,currency=dom_ccy),
    exchange_rate=1,
    transaction_currency=dom_ccy,
)

response = transaction_portfolios_api.upsert_transactions(scope=scope,
                                                          code=portfolio_code,
                                                          transaction_request=[opt_txn])

print(f"Transaction successfully updated at time: {response.version.as_at_date}")

Transaction successfully updated at time: 2024-03-06 13:57:49.710366+00:00


In [92]:
# Read fx spot rates and make datetimes timezone aware
quotes_df = pd.read_csv("data/eurusd_spot.csv")
quotes_df["Date"] = pd.to_datetime(quotes_df["Date"], dayfirst=True)
quotes_df["Date"] = quotes_df["Date"].apply(lambda x: x.replace(tzinfo=pytz.utc))
quotes_df.head()

,Date,Rate,Pair
0,2021-01-01 00:00:00+00:00,1.2215,EUR/USD
1,2021-01-04 00:00:00+00:00,1.2248,EUR/USD
2,2021-01-05 00:00:00+00:00,1.2298,EUR/USD
3,2021-01-06 00:00:00+00:00,1.2327,EUR/USD
4,2021-01-07 00:00:00+00:00,1.2272,EUR/USD


In [93]:
# Create quotes request
instrument_quotes = {
    index: lm.UpsertQuoteRequest(
        quote_id=lm.QuoteId(
            quote_series_id=lm.QuoteSeriesId(
                provider=market_supplier,
                instrument_id=row["Pair"],
                instrument_id_type="CurrencyPair",
                quote_type="Rate",
                field="mid",
            ),
            effective_at=row["Date"].isoformat(),
        ),
        metric_value=lm.MetricValue(value=row["Rate"], unit=row["Pair"]),
    )
    for index, row in quotes_df.iterrows()
}

# Upsert quotes into LUSID
response = quotes_api.upsert_quotes(
    scope=recipe_composer_scope, request_body=instrument_quotes
)

if response.failed == {}:
    print(f"Quotes successfully loaded into LUSID. {len(response.values)} quotes loaded.")
else:
    print(f"Some failures occurred during quotes upsertion, {len(response.failed)} did not get loaded into LUSID.")

Quotes successfully loaded into LUSID. 223 quotes loaded.


In [94]:
metrics = [
    lm.AggregateSpec("Instrument/default/Name", "Value"),
    lm.AggregateSpec("Instrument/default/ClientInternal", "Value"),
    lm.AggregateSpec("Valuation/Model/Name", "Value"),
    lm.AggregateSpec("Valuation/PV", "Value"),
]

valuation_request = lm.ValuationRequest(
    recipe_id=lm.ResourceId(scope=recipe_composer_scope, code=recipe_composer_code),
    metrics=metrics,
    group_by=[],
    portfolio_entity_ids=[
        lm.PortfolioEntityId(scope=scope, code=portfolio_code)
    ],
    valuation_schedule=lm.ValuationSchedule(
        effective_at = trade_date.isoformat(),
    ),
)

val_data = aggregation_api.get_valuation(valuation_request=valuation_request).data

vals_df = pd.DataFrame(val_data)

vals_df.rename(
    columns={
        "Instrument/default/Name" : "InstrumentName",
        "Instrument/default/ClientInternal" : "ClientInternal",
        "Valuation/Model/Name" : "ModelName",
        "Valuation/PV": "Value",
    },
    inplace=True,
)

display(vals_df)

,InstrumentName,ClientInternal,ModelName,Value
0,myInstrument,EURUSDOptionDemo,ConstantTimeValueOfMoney,-1.6181
1,EUR,None,ConstantTimeValueOfMoney,1.5000


As we can see above, using Recipe Composer in a Valuation can be done in the exactly same way as the Configuration Recipe is used. This is not just for Valuation, wherever a Configuration Recipe can be used so can a Recipe Composer.